## Configuration
_Initial steps to get the notebook ready to play nice with our repository. Do not delete this section._

Code formatting with [black](https://pypi.org/project/nb-black/).

In [1]:
%load_ext lab_black

In [2]:
import os
import pytz
import glob
import pathlib

this_dir = pathlib.Path(os.path.abspath(""))
data_dir = this_dir / "data"

In [3]:
import json
import requests
import pandas as pd
from slugify import slugify
from datetime import datetime

## Download

Retrieve the page

In [4]:
url = "https://services.arcgis.com/NkcnS0qk4w2wasOJ/arcgis/rest/services/COVIDCasesByCities/FeatureServer/0/query?where=0%3D0&objectIds=&time=&geometry=&geometryType=esriGeometryEnvelope&inSR=&spatialRel=esriSpatialRelIntersects&resultType=none&distance=0.0&units=esriSRUnit_Meter&returnGeodetic=false&outFields=NAME%2C+Cases&returnGeometry=true&returnCentroid=false&featureEncoding=esriDefault&multipatchOption=xyFootprint&maxAllowableOffset=&geometryPrecision=&outSR=&datumTransformation=&applyVCSProjection=false&returnIdsOnly=false&returnUniqueIdsOnly=false&returnCountOnly=false&returnExtentOnly=false&returnQueryGeometry=false&returnDistinctValues=false&cacheHint=false&orderByFields=&groupByFieldsForStatistics=&outStatistics=&having=&resultOffset=&resultRecordCount=&returnZ=false&returnM=false&returnExceededLimitFeatures=true&quantizationParameters=&sqlFormat=none&f=pgeojson&token="

In [5]:
r = requests.get(url)

In [6]:
data = r.json()

JSONDecodeError: Expecting value: line 1 column 1 (char 0)

## Parse

In [7]:
dict_list = []

In [8]:
for item in data["features"]:
    d = dict(
        county="Santa Clara",
        area=item["properties"]["NAME"],
        confirmed_cases=item["properties"]["Cases"],
    )
    if d["area"] != "UNINCORPORATED" and d["confirmed_cases"]:
        dict_list.append(d)

In [9]:
df = pd.DataFrame(dict_list)

In [10]:
df["area"] = df["area"].str.lower().str.title()

Get timestamp

In [11]:
date_url = "https://services.arcgis.com/NkcnS0qk4w2wasOJ/arcgis/rest/services/COVIDCasesByCities/FeatureServer/0?f=json"
date_r = requests.get(date_url)
date_data = date_r.json()

In [12]:
timestamp = date_data["editingInfo"]["lastEditDate"]

In [13]:
timestamp = datetime.fromtimestamp((timestamp / 1000))

In [14]:
latest_date = pd.to_datetime(timestamp).date()

In [15]:
df["county_date"] = latest_date

## Vet

In [16]:
len(df)

15

In [17]:
try:
    assert not len(df) > 15
except AssertionError:
    raise AssertionError(
        f"Santa Clara County's scraper has extra {len(export_df) - 9} rows: {list(export_df.area)}"
    )

In [18]:
try:
    assert not len(df) < 15
except AssertionError:
    raise AssertionError(
        f"Santa Clara County's scraper is missing {9 - len(export_df)} rows: {list(export_df.area)}"
    )

## Export

Set date

In [19]:
tz = pytz.timezone("America/Los_Angeles")

In [20]:
today = datetime.now(tz).date()

In [21]:
slug = "santa-clara"

In [22]:
df.to_csv(data_dir / slug / f"{today}.csv", index=False)

## Combine

In [23]:
csv_list = [
    i
    for i in glob.glob(str(data_dir / slug / "*.csv"))
    if not str(i).endswith("timeseries.csv")
]

In [24]:
df_list = []
for csv in csv_list:
    if "manual" in csv:
        df = pd.read_csv(csv, parse_dates=["date"])
    else:
        file_date = csv.split("/")[-1].replace(".csv", "")
        df = pd.read_csv(csv, parse_dates=["county_date"])
        df["date"] = file_date
    df_list.append(df)

In [25]:
df = pd.concat(df_list).sort_values(["date", "area"])

In [26]:
df.to_csv(data_dir / slug / "timeseries.csv", index=False)